In [16]:
import pandas as pd
# import bamboolib as bam
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

sample_surveys = './sample/Customer Satisfaction Surveys (3099) (5) (1).xlsx'
sample_fundings = './sample/Funding By Referral Source (2006) (9).xlsx'

teams = {
    'Team 1': [
        'cherylbosch',
        'coleenkellner',
        'brandonkiefer'
    ],
    'Team 2': [
        'jeannefigdore',
        'karenpatterson',
        'sonjaharwood'
    ],
    'Team 3': [
        'lauriebullock',
        'caribaker',
        'debbiemcintyre'
    ]
}

def read_processor_details(file): 
    df = pd.read_excel(file, skiprows=5, sheet_name='Processor Details')
    df.rename(columns={'Processor To Loan Number': 'Loan Number'}, inplace=True)

    selected_processor = ''
    ammended_rows = []
    for _,row in df.iterrows():
        if str.isalpha(row['Loan Number'].replace(' ', '')):
            processor = row['Loan Number'].replace(' ', '')
            selected_processor = processor
        if pd.to_numeric(row['Loan Number'], errors='coerce') > 1000:
            row = row.to_dict()
            row['Processor'] = selected_processor
            ammended_rows.append(row)
    
    df = pd.DataFrame(ammended_rows)
    df['Loan Number'] = df['Loan Number'].apply(pd.to_numeric)

    def assign_team(processor):
        team = [team for team in teams if processor in teams[team]]
        if any(team):
            return team[0]
        return ''
    
    df['Team'] = df['Processor'].apply(assign_team)
    df.set_index('Loan Number', inplace=True)

    return df

def read_fundings(file):
    df = pd.read_excel(sample_fundings, sheet_name='Details', skiprows=4)
    df.columns = [col.split('\n')[0] for col in df.columns]
    df['Loan Number'] = df['Loan Number'].apply(pd.to_numeric)
    df.set_index('Loan Number', inplace=True)
    return df

def load_csat_data(csat_file, fundings_file):
    fundings = read_fundings(sample_fundings)
    csat_surveys = read_processor_details(sample_surveys)
    df = csat_surveys.join(fundings[['Funded Month', 'Loan Officer', 'Referral Source', 'Referral Name']])
    df = df.reset_index()
    df = df[['Loan Number', 'Funded Month', 'Processor', 'Team', 'Loan Officer', 'Top Box CSAT Feedback', 'Top Box Recommend Feedback', 'Top Box Repeat Feedback', 'Top Box LOCSAT Feedback', 'Top Box LPCSAT Feedback', 'Top Box Closing Feedback', 'Strength Feedback', 'Recommendation Feedback', 'Notes Feedback', 'Borrower Last Name', 'Borrower First Name', 'Borrower Email', 'Referral Source', 'Referral Name'] + ['LOS System', 'Top Two CSAT Feedback', 'Top Two Recommend Feedback', 'Top Two Repeat Feedback', 'Top Two LOCSAT Feedback', 'Top Two LPCSAT Feedback', 'Top Two Closing Feedback']]
    df = df.dropna(how='all', axis=1)
    df = df.drop(columns=['LOS System'])
    return df

def create_word_cloud(df: 'pd.dataframe', col_name: str):
    text = df[col_name].dropna().values
    wordcloud = WordCloud(
        width=3000,
        height=2000,
        background_color='white',
        stopwords=STOPWORDS).generate(str(text))
    fig = plt.figure(
        figsize=(40, 30),
        facecolor='k',
        edgecolor='white')
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

class TeamReport(object):
    def __init__(self, 
                team_name, 
                surveys_path=sample_surveys, 
                fundings_path=sample_fundings):
        self.team_name = team_name
        self.surveys_path = surveys_path
        self.fundings_path = fundings_path
        self.init()
        
    def init(self):
        df = load_csat_data(self.surveys_path, self.fundings_path)
        df = df[df['Team'] == self.team_name]
        self.df = df
    
    def strength_cloud(self):
        create_word_cloud(self.df, 'Strength Feedback')
    
    def recommendation_cloud(self):
        create_word_cloud(self.df, 'Recommendation Feedback')

################## TO RUN ################
# df = load_csat_data(sample_surveys, sample_fundings)
team_report = TeamReport('Team 1')


In [18]:
df = load_csat_data(sample_surveys, sample_fundings)

In [19]:
df

,Loan Number,Funded Month,Processor,Team,Loan Officer,Top Box CSAT Feedback,Top Box Recommend Feedback,Top Box Repeat Feedback,Top Box LOCSAT Feedback,Top Box LPCSAT Feedback,Top Box Closing Feedback,Strength Feedback,Recommendation Feedback,Borrower Last Name,Borrower First Name,Borrower Email,Referral Source,Referral Name
0,280001523779,2021-09-01,brandonkiefer,Team 1,Jimmy Bohler,5,5,5,5,5,5,NaN,NaN,Pope,Jessica Lauren,Jessica.Pope@SupremeLending.com,*no ASA account associated,Supreme Lending Employee
1,290001335943,2021-05-01,brandonkiefer,Team 1,Bart Maldonado,5,5,5,5,5,5,Communication,You are all amazing!,Daniel,Ashley Pate,adaniel@sandersteamrealty.com,Katrina Sells Georgia,NaN
2,290001336040,2021-05-01,brandonkiefer,Team 1,Bart Maldonado,5,5,5,5,5,5,NaN,NaN,Diersen,Matthew,matthewdiersen@gmail.com,*no ASA account associated,Aliaa Alsifadi
3,290001336810,2021-04-01,brandonkiefer,Team 1,Bart Maldonado,5,5,5,5,5,5,NaN,NaN,Sherard,Anthony,Asherard1216@gmail.com,*no ASA account associated,Tavaras Jones
4,290001336830,2021-04-01,brandonkiefer,Team 1,Bart Maldonado,5,5,5,5,5,5,NaN,NaN,Amouzou,Akouvi Akpedje,aamouzou82@gmail.com,The Radtke Team,Brian Smith
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,290210286574,2021-03-01,sydneydahle,,Darren Griffin,0,0,0,0,0,0,NaN,NaN,Degirmenci,Mufit Cem,mcdegirmenci11@gmail.com,NaN,Joe Smith
5847,290210286606,2021-03-01,sydneydahle,,Kimberly Nehiley,0,0,0,0,0,0,NaN,NaN,Brenner,Eric,eric@authenteak.com,NaN,NaN
5848,290210287144,2021-03-01,sydneydahle,,Kimberly Nehiley,5,5,5,5,5,5,NaN,NaN,McLendon,Cherise,cjmclendon22@gmail.com,NaN,NaN
5849,294201275605,2021-01-01,sydneydahle,,Joan Quinn,5,5,5,5,5,5,NaN,NaN,Haefner,Daniel E,danhef@bellsouth.net,NaN,NaN


In [26]:
import numpy as np
df['Top Box CSAT Feedback'].filter

TypeError: <lambda>() got an unexpected keyword argument 'axis'